In [17]:
!pip install boruta category_encoders xgboost catboost
!pip uninstall -y scikit-learn imbalanced-learn

!pip install scikit-learn==1.4.2 imbalanced-learn==0.12.0


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler, SMOTENC
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import CondensedNearestNeighbour, TomekLinks, RandomUnderSampler
from boruta import BorutaPy
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss
from imblearn.under_sampling import TomekLinks

from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss, TomekLinks
from imblearn.combine import SMOTETomek, SMOTEENN

Found existing installation: scikit-learn 1.4.2
Uninstalling scikit-learn-1.4.2:
  Successfully uninstalled scikit-learn-1.4.2
Found existing installation: imbalanced-learn 0.12.0
Uninstalling imbalanced-learn-0.12.0:
  Successfully uninstalled imbalanced-learn-0.12.0
  Using cached scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached imbalanced_learn-0.12.0-py3-none-any.whl.metadata (8.2 kB)
Using cached scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
Using cached imbalanced_learn-0.12.0-py3-none-any.whl (257 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [18]:
# ==============================
# Combined Feature Engineering Pipeline
# ==============================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ------------------------------
# Step 1: Load dataset
# ------------------------------
df = pd.read_csv("/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv")
df.fillna("None", inplace=True)





# ------------------------------
# Step 2: Basic preprocessing
# ------------------------------
# Split Blood Pressure
df[['Systolic BP', 'Diastolic BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df.drop(['Person ID', 'Blood Pressure'], axis=1, inplace=True)

# Group rare occupations
df['Occupation'] = df['Occupation'].replace(['Manager', 'Sales Representative', 'Scientist', 'Software Engineer'], 'Other')

# Map BMI categories to numeric
df['BMI Category'] = df['BMI Category'].replace({'Normal': 22, 'Normal Weight': 22, 'Overweight': 27, 'Obese': 30})

# Label encode categorical columns
label_encoder = LabelEncoder()
for col in ['Gender', 'Sleep Disorder']:
    df[col] = label_encoder.fit_transform(df[col])

# One-hot encode Occupation
df = pd.get_dummies(df, columns=['Occupation'], drop_first=False)

# ------------------------------
# Step 3: Feature Engineering
# ------------------------------
epsilon = 1e-6

# ---- Ratio & difference features ----
df['Sleep_Heart_ratio'] = df['Sleep Duration'] / (df['Heart Rate'] + epsilon)
df['Sleep_Steps_ratio'] = df['Sleep Duration'] / (df['Daily Steps'] + epsilon)
df['Stress_Activity_ratio'] = df['Stress Level'] / (df['Physical Activity Level'] + epsilon)
df['Heart_Sleep_diff'] = df['Heart Rate'] - df['Sleep Duration']



# ---- Aggregation features ----
df['Total_Activity'] = df['Physical Activity Level'] + df['Daily Steps']
df['Stress_per_Activity'] = df['Stress Level'] / (df['Total_Activity'] + epsilon)

# ---- Interaction features ----
df['Sleep_Stress_interaction'] = df['Sleep Duration'] * df['Stress Level']
df['BMI_Activity_interaction'] = df['BMI Category'] * df['Physical Activity Level']

# ---- Categorical interaction features ----
df['Gender_Occupation'] = df['Gender'].astype(str) + "_" + df['Occupation_Other'].astype(str)
df = pd.get_dummies(df, columns=['Gender_Occupation'], drop_first=False)



df = pd.read_csv("/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv")
df.fillna("None", inplace=True)

# Dividing Blood Pressure into Systolic and Diastolic BP
df[['Systolic BP', 'Diastolic BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df.drop(['Person ID', 'Blood Pressure'], axis=1, inplace=True)

# Labeling less number of careers as other
df['Occupation'] = df['Occupation'].replace(['Manager', 'Sales Representative', 'Scientist', 'Software Engineer'], 'Other')

# Adding the average BMI for the range
df['BMI Category'] = df['BMI Category'].replace({'Normal':22, 'Normal Weight':22, 'Overweight':27, 'Obese':30})

# Creating Interaction features
df['Stress_sleep_interaction'] = df['Stress Level'] / df['Quality of Sleep']
df['BMI_Activity'] = df['BMI Category'] * df['Physical Activity Level']
df['Sleep_Heart_ratio'] = df['Sleep Duration'] / df['Heart Rate']
df['Sleep_Steps_ratio'] = df['Sleep Duration'] / df['Daily Steps']
df['Sleep_Stress_ratio'] = df['Sleep Duration'] / df['Stress Level']

df = pd.get_dummies(df, columns=['Occupation'], drop_first=False)

label_encoder = LabelEncoder()
columns = ['Gender', 'Sleep Disorder']
for col in columns:
  df[col] = label_encoder.fit_transform(df[col])


'''


num_col = ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Stress Level', 'Stress_sleep_interaction',
          'Sleep_Heart_ratio', 'Sleep_Steps_ratio', 'Sleep_Stress_ratio', 'Heart Rate', 'Daily Steps',
           'Systolic BP', 'Diastolic BP']

Q1 = df[num_col].quantile(0.25)
Q3 = df[num_col].quantile(0.75)
IQR = Q3 - Q1
# getting rid of the outliers
df = df[~((df[num_col] < (Q1 - 1.4 * IQR)) | (df[num_col] > (Q3 + 1.4 * IQR))).any(axis=1)]

print(df.shape)


'''
# ------------------------------
# Step 4: Prepare features & target
# ------------------------------
X = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']

# ------------------------------
# Step 5: Train-test split
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

'''
# Get only the numeric columns
num_col = X_train.select_dtypes(include='number').columns

Q1 = X_train[num_col].quantile(0.25)
Q3 = X_train[num_col].quantile(0.75)
IQR = Q3 - Q1

# Remove outliers from X_train
X_train = X_train[~((X_train[num_col] < (Q1 - 3 * IQR)) | (X_train[num_col] > (Q3 + 3* IQR))).any(axis=1)]
y_train = y_train[X_train.index]
'''


"\n# Get only the numeric columns\nnum_col = X_train.select_dtypes(include='number').columns\n\nQ1 = X_train[num_col].quantile(0.25)\nQ3 = X_train[num_col].quantile(0.75)\nIQR = Q3 - Q1\n\n# Remove outliers from X_train\nX_train = X_train[~((X_train[num_col] < (Q1 - 3 * IQR)) | (X_train[num_col] > (Q3 + 3* IQR))).any(axis=1)]\ny_train = y_train[X_train.index]\n"

In [19]:
'''

import numpy as np
import pandas as pd

# Get only numeric columns
num_col = X_train.select_dtypes(include='number').columns

# Compute IQR
Q1 = X_train[num_col].quantile(0.25)
Q3 = X_train[num_col].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers (e.g., 3*IQR)
upper_bound = Q3 + 3 * IQR
lower_bound = Q1 - 3 * IQR

# Sample some rows (you can also target specific values/classes if needed)
n_outliers_to_add = 100  # or int(0.05 * len(X_train)) for 5% more
sampled_rows = X_train.sample(n=n_outliers_to_add, random_state=42)

# Apply noise to numeric columns to push them into outlier space
outliers = sampled_rows.copy()
for col in num_col:
    direction = np.random.choice([-1, 1], size=n_outliers_to_add)
    magnitude = np.random.uniform(3, 5, size=n_outliers_to_add)  # stronger than 3*IQR
    outliers[col] += direction * magnitude * IQR[col]

# Append to original data
X_train_augmented = pd.concat([X_train, outliers], axis=0)
y_train_augmented = pd.concat([y_train, y_train.loc[sampled_rows.index]], axis=0)


'''

"\n\nimport numpy as np\nimport pandas as pd\n\n# Get only numeric columns\nnum_col = X_train.select_dtypes(include='number').columns\n\n# Compute IQR\nQ1 = X_train[num_col].quantile(0.25)\nQ3 = X_train[num_col].quantile(0.75)\nIQR = Q3 - Q1\n\n# Define bounds for outliers (e.g., 3*IQR)\nupper_bound = Q3 + 3 * IQR\nlower_bound = Q1 - 3 * IQR\n\n# Sample some rows (you can also target specific values/classes if needed)\nn_outliers_to_add = 100  # or int(0.05 * len(X_train)) for 5% more\nsampled_rows = X_train.sample(n=n_outliers_to_add, random_state=42)\n\n# Apply noise to numeric columns to push them into outlier space\noutliers = sampled_rows.copy()\nfor col in num_col:\n    direction = np.random.choice([-1, 1], size=n_outliers_to_add)\n    magnitude = np.random.uniform(3, 5, size=n_outliers_to_add)  # stronger than 3*IQR\n    outliers[col] += direction * magnitude * IQR[col]\n\n# Append to original data\nX_train_augmented = pd.concat([X_train, outliers], axis=0)\ny_train_augmented = 

In [20]:
df.shape

(374, 25)

In [21]:
# -----------------------------
# RandomForest classifier (for Boruta feature selection)
# -----------------------------
scaler = MinMaxScaler()
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Normalize data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# -----------------------------
# Boruta Feature Selection
# -----------------------------
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=0, random_state=42)

X_train_boruta = boruta_selector.fit_transform(X_train_normalized, y_train)
X_test_boruta = boruta_selector.transform(X_test_normalized)

# -----------------------------
# Autoencoder Architecture
# -----------------------------
n_features = X_train_boruta.shape[1]
input_layer = Input(shape=(n_features,))

# Encoder
encoded = Dense(32, activation='relu')(input_layer)
bottleneck = Dense(16, activation='relu')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(bottleneck)
output_layer = Dense(n_features, activation='sigmoid')(decoded)

# Full autoencoder model
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train_boruta, X_train_boruta, epochs=10, batch_size=32, verbose=0)

# Encoder model to extract bottleneck features
encoder = Model(inputs=input_layer, outputs=bottleneck)

# Transform data
X_train_encoded = encoder.predict(X_train_boruta)
X_test_encoded = encoder.predict(X_test_boruta)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [22]:
scaler = RobustScaler()
X_train_robust = scaler.fit_transform(X_train)
X_test_robust = scaler.transform(X_test)

# Feature selection using Mutual Information
mi = SelectKBest(score_func=mutual_info_classif, k=5)
X_train_mi = mi.fit_transform(X_train_robust, y_train)
X_test_mi = mi.transform(X_test_robust)

# Applying Linear Discriminant Analysis (LDA)
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_mi, y_train)
X_test_lda = lda.transform(X_test_mi)

In [23]:
from sklearn.preprocessing import PowerTransformer
scaler = PowerTransformer(method='yeo-johnson')
X_train_power = scaler.fit_transform(X_train)
X_test_power = scaler.transform(X_test)


Boruta + SMOTE + Tomek


PowerTransformer + SMOTE


MI + SMOTE + Tomek


LDA


MI


In [24]:

# Boruta + SMOTE+Tomek

smotetomek_boruta = SMOTETomek(random_state=42)
X_train_boruta_smotetomek, y_train_boruta_smotetomek = smotetomek_boruta.fit_resample(X_train_boruta, y_train)
X_test_boruta_smotetomek = X_test_boruta.copy()


# PowerTransformer + SMOTE
smote_power = SMOTE(random_state=42, k_neighbors=5)
X_train_power_smote, y_train_power_smote = smote_power.fit_resample(X_train_power, y_train)
X_test_power_smote = X_test_power.copy()


# MI + SMOTE+Tomek
smotetomek_mi = SMOTETomek(random_state=42)
X_train_mi_smotetomek, y_train_mi_smotetomek = smotetomek_mi.fit_resample(X_train_mi, y_train)
X_test_mi_smotetomek = X_test_mi.copy()



# MI
mi = SelectKBest(score_func=mutual_info_classif, k=5)
X_train_mi = mi.fit_transform(X_train_robust, y_train)
X_test_mi = mi.transform(X_test_robust)

# LDA
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_mi, y_train)
X_test_lda = lda.transform(X_test_mi)

# training individual model

In [25]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier, 
                               ExtraTreesClassifier, GradientBoostingClassifier)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                              f1_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Assuming you have X_train, X_test, y_train, y_test already loaded
# You need to have the base preprocessing done before running configurations

# ==============================================================================
# STEP 1: PREPARE ALL 5 CONFIGURATIONS
# ==============================================================================

print("Preparing all configurations...")

# Assuming you have X_train_robust and X_test_robust ready (with RobustScaler applied)
# If not, create them first:
# scaler = RobustScaler()
# X_train_robust = scaler.fit_transform(X_train)
# X_test_robust = scaler.transform(X_test)

# Configuration 1: Boruta + SMOTE+Tomek
# Assuming X_train_boruta and X_test_boruta are already prepared from Boruta selection
smotetomek_boruta = SMOTETomek(random_state=42)
X_train_boruta_smotetomek, y_train_boruta_smotetomek = smotetomek_boruta.fit_resample(X_train_boruta, y_train)
X_test_boruta_smotetomek = X_test_boruta.copy()
y_test_boruta_smotetomek = y_test.copy()

# Configuration 2: PowerTransformer + SMOTE
# Assuming X_train_power and X_test_power are already prepared
smote_power = SMOTE(random_state=42, k_neighbors=5)
X_train_power_smote, y_train_power_smote = smote_power.fit_resample(X_train_power, y_train)
X_test_power_smote = X_test_power.copy()
y_test_power_smote = y_test.copy()

# Configuration 3: MI + SMOTE+Tomek
# First apply MI selection
mi = SelectKBest(score_func=mutual_info_classif, k=5)
X_train_mi = mi.fit_transform(X_train_robust, y_train)
X_test_mi = mi.transform(X_test_robust)

smotetomek_mi = SMOTETomek(random_state=42)
X_train_mi_smotetomek, y_train_mi_smotetomek = smotetomek_mi.fit_resample(X_train_mi, y_train)
X_test_mi_smotetomek = X_test_mi.copy()
y_test_mi_smotetomek = y_test.copy()

# Configuration 4: MI only (without SMOTE)
# Already created above: X_train_mi, X_test_mi
y_train_mi = y_train.copy()
y_test_mi = y_test.copy()

# Configuration 5: LDA
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_mi, y_train)
X_test_lda = lda.transform(X_test_mi)
y_train_lda = y_train.copy()
y_test_lda = y_test.copy()

print("All configurations prepared!\n")

# ==============================================================================
# STEP 2: DEFINE CLASSIFIERS AND HELPER FUNCTIONS
# ==============================================================================

# Define all classifiers - CREATE NEW INSTANCES FOR EACH TRAINING
from sklearn.linear_model import LogisticRegression

def get_classifiers():
    return {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
        'MLPClassifier': MLPClassifier(random_state=42, max_iter=1000),
        'SVC': SVC(random_state=42, probability=True),
        'AdaBoostClassifier': AdaBoostClassifier(random_state=42, algorithm='SAMME'),
        'RandomForestClassifier': RandomForestClassifier(random_state=42, n_estimators=100),
        'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42, n_estimators=100),
        'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
        'XGBClassifier': XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False),
        'LGBMClassifier': LGBMClassifier(random_state=42, verbose=-1),
        'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42)
    }

# Function to calculate metrics
def calculate_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate classification metrics"""
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'Recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }
    
    # Add ROC-AUC if probability predictions are available
    if y_pred_proba is not None:
        try:
            if len(np.unique(y_true)) == 2:  # Binary classification
                metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba[:, 1])
            else:  # Multi-class
                metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba, 
                                                    multi_class='ovr', average='weighted')
        except Exception as e:
            metrics['ROC-AUC'] = 'N/A'
    else:
        metrics['ROC-AUC'] = 'N/A'
    
    return metrics

# Function to train and evaluate a model
def train_and_evaluate(model, X_train, y_train, X_test, y_test, model_name, config_name):
    """Train model and return metrics"""
    try:
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Get probability predictions if available
        try:
            y_pred_proba = model.predict_proba(X_test)
        except:
            y_pred_proba = None
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
        metrics['Model'] = model_name
        metrics['Configuration'] = config_name
        
        return metrics
    
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        return {
            'Model': model_name,
            'Configuration': config_name,
            'Accuracy': 0.0,
            'Precision': 0.0,
            'Recall': 0.0,
            'F1-Score': 0.0,
            'ROC-AUC': 'Error'
        }

# ==============================================================================
# STEP 3: TRAIN ALL MODELS ON ALL CONFIGURATIONS
# ==============================================================================

# Store all configurations
configurations = [
    ('Boruta + SMOTE+Tomek', X_train_boruta_smotetomek, y_train_boruta_smotetomek, 
     X_test_boruta_smotetomek, y_test_boruta_smotetomek),
    ('PowerTransformer + SMOTE', X_train_power_smote, y_train_power_smote, 
     X_test_power_smote, y_test_power_smote),
    ('MI + SMOTE+Tomek', X_train_mi_smotetomek, y_train_mi_smotetomek, 
     X_test_mi_smotetomek, y_test_mi_smotetomek),
    ('MI', X_train_mi, y_train_mi, X_test_mi, y_test_mi),
    ('LDA', X_train_lda, y_train_lda, X_test_lda, y_test_lda)
]

# Dictionary to store all results
all_results = []

# Train all models on all configurations
for config_name, X_tr, y_tr, X_te, y_te in configurations:
    print("="*80)
    print(f"CONFIGURATION: {config_name}")
    print("="*80)
    print(f"Training shape: {X_tr.shape}, Test shape: {X_te.shape}")
    print()
    
    classifiers = get_classifiers()  # Get fresh instances
    
    for model_name, model in classifiers.items():
        print(f"Training {model_name}...", end=' ')
        
        metrics = train_and_evaluate(
            model, X_tr, y_tr, X_te, y_te, model_name, config_name
        )
        all_results.append(metrics)
        
        # Display metrics inline
        print(f"✓ Acc: {metrics['Accuracy']:.4f} | Prec: {metrics['Precision']:.4f} | " +
              f"Rec: {metrics['Recall']:.4f} | F1: {metrics['F1-Score']:.4f} | " +
              f"AUC: {metrics['ROC-AUC']}")
    
    print()

# ==============================================================================
# STEP 4: DISPLAY ALL RESULTS
# ==============================================================================

# Create comprehensive results DataFrame
results_df = pd.DataFrame(all_results)

# Display summary table sorted by accuracy
print("\n" + "="*80)
print("📊 ALL RESULTS SORTED BY ACCURACY (HIGHEST TO LOWEST)")
print("="*80)
results_sorted = results_df.sort_values('Accuracy', ascending=False).reset_index(drop=True)
results_sorted.index = results_sorted.index + 1  # Start index from 1

# Format the display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print(results_sorted.to_string())

# Top 10 models
print("\n" + "="*80)
print("🏆 TOP 10 MODEL-CONFIGURATION COMBINATIONS")
print("="*80)
print(results_sorted.head(10).to_string())

# Best model per configuration
print("\n" + "="*80)
print("🥇 BEST MODEL PER CONFIGURATION")
print("="*80)
for config in configurations:
    config_name = config[0]
    config_results = results_df[results_df['Configuration'] == config_name]
    best_model = config_results.loc[config_results['Accuracy'].idxmax()]
    print(f"\n{config_name}:")
    print(f"  Model:      {best_model['Model']}")
    print(f"  Accuracy:   {best_model['Accuracy']:.4f}")
    print(f"  Precision:  {best_model['Precision']:.4f}")
    print(f"  Recall:     {best_model['Recall']:.4f}")
    print(f"  F1-Score:   {best_model['F1-Score']:.4f}")
    print(f"  ROC-AUC:    {best_model['ROC-AUC']}")

# Best configuration per model
print("\n" + "="*80)
print("🎯 BEST CONFIGURATION PER MODEL")
print("="*80)
model_names = results_df['Model'].unique()
for model_name in sorted(model_names):
    model_results = results_df[results_df['Model'] == model_name]
    best_config = model_results.loc[model_results['Accuracy'].idxmax()]
    print(f"\n{model_name}:")
    print(f"  Configuration: {best_config['Configuration']}")
    print(f"  Accuracy:      {best_config['Accuracy']:.4f}")
    print(f"  F1-Score:      {best_config['F1-Score']:.4f}")
    print(f"  ROC-AUC:       {best_config['ROC-AUC']}")

# Overall best model
print("\n" + "="*80)
print("🌟 OVERALL BEST MODEL-CONFIGURATION COMBINATION")
print("="*80)
best_overall = results_df.loc[results_df['Accuracy'].idxmax()]
print(f"Model:         {best_overall['Model']}")
print(f"Configuration: {best_overall['Configuration']}")
print(f"Accuracy:      {best_overall['Accuracy']:.4f}")
print(f"Precision:     {best_overall['Precision']:.4f}")
print(f"Recall:        {best_overall['Recall']:.4f}")
print(f"F1-Score:      {best_overall['F1-Score']:.4f}")
print(f"ROC-AUC:       {best_overall['ROC-AUC']}")

# Create pivot tables
print("\n" + "="*80)
print("📈 PIVOT TABLE: ACCURACY BY MODEL AND CONFIGURATION")
print("="*80)
pivot_accuracy = results_df.pivot(index='Model', columns='Configuration', values='Accuracy')
print(pivot_accuracy.to_string())

print("\n" + "="*80)
print("📈 PIVOT TABLE: F1-SCORE BY MODEL AND CONFIGURATION")
print("="*80)
pivot_f1 = results_df.pivot(index='Model', columns='Configuration', values='F1-Score')
print(pivot_f1.to_string())

# Export results
results_df.to_csv('model_comparison_results.csv', index=False)
print("\n" + "="*80)
print("💾 Results exported to 'model_comparison_results.csv'")
print("="*80)

# Summary statistics
print("\n" + "="*80)
print("📊 SUMMARY STATISTICS")
print("="*80)
print(f"Total experiments run: {len(results_df)}")
print(f"Number of models: {len(model_names)}")
print(f"Number of configurations: {len(configurations)}")
print(f"\nAccuracy Statistics:")
print(f"  Mean:    {results_df['Accuracy'].mean():.4f}")
print(f"  Median:  {results_df['Accuracy'].median():.4f}")
print(f"  Std Dev: {results_df['Accuracy'].std():.4f}")
print(f"  Min:     {results_df['Accuracy'].min():.4f}")
print(f"  Max:     {results_df['Accuracy'].max():.4f}")

Preparing all configurations...
All configurations prepared!

CONFIGURATION: Boruta + SMOTE+Tomek
Training shape: (519, 10), Test shape: (75, 10)

Training LogisticRegression... ✓ Acc: 0.9467 | Prec: 0.9499 | Rec: 0.9467 | F1: 0.9476 | AUC: 0.9763151691877772
Training MLPClassifier... ✓ Acc: 0.9467 | Prec: 0.9573 | Rec: 0.9467 | F1: 0.9484 | AUC: 0.9865429803778628
Training SVC... ✓ Acc: 0.9333 | Prec: 0.9419 | Rec: 0.9333 | F1: 0.9341 | AUC: 0.9788162323066641
Training AdaBoostClassifier... ✓ Acc: 0.9067 | Prec: 0.9065 | Rec: 0.9067 | F1: 0.9062 | AUC: 0.9820314683190571
Training RandomForestClassifier... ✓ Acc: 0.9600 | Prec: 0.9606 | Rec: 0.9600 | F1: 0.9599 | AUC: 0.9941694915254238
Training ExtraTreesClassifier... ✓ Acc: 0.9333 | Prec: 0.9341 | Rec: 0.9333 | F1: 0.9333 | AUC: 0.9768079096045199
Training GradientBoostingClassifier... ✓ Acc: 0.9600 | Prec: 0.9606 | Rec: 0.9600 | F1: 0.9599 | AUC: 0.9824745762711863
Training XGBClassifier... ✓ Acc: 0.9333 | Prec: 0.9341 | Rec: 0.9333

# Stacking,voting and ensembling

In [26]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier, 
                               ExtraTreesClassifier, GradientBoostingClassifier,
                               StackingClassifier, VotingClassifier, BaggingClassifier)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                              f1_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# STEP 1: PREPARE ALL 5 CONFIGURATIONS
# ==============================================================================

print("="*80)
print("PREPARING ALL CONFIGURATIONS")
print("="*80)

# Configuration 1: Boruta + SMOTE+Tomek
print("1. Boruta + SMOTE+Tomek...")
smotetomek_boruta = SMOTETomek(random_state=42)
X_train_boruta_smotetomek, y_train_boruta_smotetomek = smotetomek_boruta.fit_resample(X_train_boruta, y_train)
X_test_boruta_smotetomek = X_test_boruta.copy()
y_test_boruta_smotetomek = y_test.copy()

# Configuration 2: PowerTransformer + SMOTE
print("2. PowerTransformer + SMOTE...")
smote_power = SMOTE(random_state=42, k_neighbors=5)
X_train_power_smote, y_train_power_smote = smote_power.fit_resample(X_train_power, y_train)
X_test_power_smote = X_test_power.copy()
y_test_power_smote = y_test.copy()

# Configuration 3: MI + SMOTE+Tomek
print("3. MI + SMOTE+Tomek...")
mi = SelectKBest(score_func=mutual_info_classif, k=5)
X_train_mi = mi.fit_transform(X_train_robust, y_train)
X_test_mi = mi.transform(X_test_robust)

smotetomek_mi = SMOTETomek(random_state=42)
X_train_mi_smotetomek, y_train_mi_smotetomek = smotetomek_mi.fit_resample(X_train_mi, y_train)
X_test_mi_smotetomek = X_test_mi.copy()
y_test_mi_smotetomek = y_test.copy()

# Configuration 4: MI only
print("4. MI only...")
y_train_mi = y_train.copy()
y_test_mi = y_test.copy()

# Configuration 5: LDA
print("5. LDA...")
lda = LinearDiscriminantAnalysis(n_components=2)
X_train_lda = lda.fit_transform(X_train_mi, y_train)
X_test_lda = lda.transform(X_test_mi)
y_train_lda = y_train.copy()
y_test_lda = y_test.copy()

print("\n✓ All configurations prepared!\n")

# ==============================================================================
# STEP 2: DEFINE BASE CLASSIFIERS
# ==============================================================================

def get_base_classifiers():
    """Returns dictionary of base classifiers"""
    return {
        'MLPClassifier': MLPClassifier(random_state=42, max_iter=1000),
        'SVC': SVC(random_state=42, probability=True),
        'AdaBoostClassifier': AdaBoostClassifier(random_state=42, algorithm='SAMME'),
        'RandomForestClassifier': RandomForestClassifier(random_state=42, n_estimators=100),
        'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42, n_estimators=100),
        'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
        'XGBClassifier': XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False),
        'LGBMClassifier': LGBMClassifier(random_state=42, verbose=-1),
        'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42)
    }

# ==============================================================================
# STEP 3: DEFINE ENSEMBLE CLASSIFIERS
# ==============================================================================

def get_ensemble_classifiers():
    """Returns dictionary of ensemble classifiers using improved base estimators"""

    # Diverse base estimators for ensembles
    base_estimators = [
        ('xgb', XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False, n_jobs=-1)),
        ('lgbm', LGBMClassifier(random_state=42, verbose=-1, n_jobs=-1)),
        ('et', ExtraTreesClassifier(random_state=42, n_estimators=100, n_jobs=-1)),
        ('gb', GradientBoostingClassifier(random_state=42, n_estimators=100)),
        ('svc', SVC(probability=True, kernel='rbf', C=1, gamma='scale', random_state=42))
    ]

    # Stacking Classifier with Logistic Regression meta-model
    stacking = StackingClassifier(
        estimators=base_estimators,
        final_estimator=LogisticRegression(
            random_state=42,
            max_iter=2000,
            solver='lbfgs',
            n_jobs=-1
        ),
        cv=5,
        n_jobs=-1,
        passthrough=True
    )

    # Voting Classifier (Soft voting for probability averaging)
    voting = VotingClassifier(
        estimators=base_estimators,
        voting='soft',
        n_jobs=-1
    )

    # Bagging Ensemble using shallow Decision Trees (better for bagging)
    bagging = BaggingClassifier(
        estimator=DecisionTreeClassifier(
            max_depth=5,
            min_samples_split=4,
            random_state=42
        ),
        n_estimators=50,
        max_samples=0.8,
        max_features=0.8,
        bootstrap=True,
        n_jobs=-1,
        random_state=42
    )

    return {
        'StackingClassifier': stacking,
        'VotingClassifier': voting,
        'BaggingEnsemble': bagging
    }


# ==============================================================================
# STEP 4: HELPER FUNCTIONS
# ==============================================================================

def calculate_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate classification metrics"""
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'Recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }
    
    if y_pred_proba is not None:
        try:
            if len(np.unique(y_true)) == 2:
                metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba[:, 1])
            else:
                metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba, 
                                                    multi_class='ovr', average='weighted')
        except:
            metrics['ROC-AUC'] = 'N/A'
    else:
        metrics['ROC-AUC'] = 'N/A'
    
    return metrics

def train_and_evaluate(model, X_train, y_train, X_test, y_test, model_name, config_name):
    """Train model and return metrics"""
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        try:
            y_pred_proba = model.predict_proba(X_test)
        except:
            y_pred_proba = None
        
        metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
        metrics['Model'] = model_name
        metrics['Configuration'] = config_name
        metrics['Model_Type'] = 'Ensemble' if model_name in ['StackingClassifier', 'VotingClassifier', 'BaggingEnsemble'] else 'Base'
        
        return metrics
    
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        return {
            'Model': model_name,
            'Configuration': config_name,
            'Model_Type': 'Ensemble' if model_name in ['StackingClassifier', 'VotingClassifier', 'BaggingEnsemble'] else 'Base',
            'Accuracy': 0.0,
            'Precision': 0.0,
            'Recall': 0.0,
            'F1-Score': 0.0,
            'ROC-AUC': 'Error'
        }

# ==============================================================================
# STEP 5: PREPARE CONFIGURATIONS LIST
# ==============================================================================

configurations = [
    ('Boruta + SMOTE+Tomek', X_train_boruta_smotetomek, y_train_boruta_smotetomek, 
     X_test_boruta_smotetomek, y_test_boruta_smotetomek),
    ('PowerTransformer + SMOTE', X_train_power_smote, y_train_power_smote, 
     X_test_power_smote, y_test_power_smote),
    ('MI + SMOTE+Tomek', X_train_mi_smotetomek, y_train_mi_smotetomek, 
     X_test_mi_smotetomek, y_test_mi_smotetomek),
    ('MI', X_train_mi, y_train_mi, X_test_mi, y_test_mi),
    ('LDA', X_train_lda, y_train_lda, X_test_lda, y_test_lda)
]

# ==============================================================================
# STEP 6: TRAIN ALL BASE MODELS ON ALL CONFIGURATIONS
# ==============================================================================

all_results = []

print("="*80)
print("PHASE 1: TRAINING BASE MODELS")
print("="*80)
print()

for config_name, X_tr, y_tr, X_te, y_te in configurations:
    print("="*80)
    print(f"CONFIGURATION: {config_name}")
    print("="*80)
    print(f"Training shape: {X_tr.shape}, Test shape: {X_te.shape}")
    print()
    
    base_classifiers = get_base_classifiers()
    
    for model_name, model in base_classifiers.items():
        print(f"  Training {model_name}...", end=' ')
        
        metrics = train_and_evaluate(
            model, X_tr, y_tr, X_te, y_te, model_name, config_name
        )
        all_results.append(metrics)
        
        print(f"✓ Acc: {metrics['Accuracy']:.4f} | F1: {metrics['F1-Score']:.4f}")
    
    print()

# ==============================================================================
# STEP 7: TRAIN ALL ENSEMBLE MODELS ON ALL CONFIGURATIONS
# ==============================================================================

print("\n" + "="*80)
print("PHASE 2: TRAINING ENSEMBLE MODELS")
print("="*80)
print()

for config_name, X_tr, y_tr, X_te, y_te in configurations:
    print("="*80)
    print(f"CONFIGURATION: {config_name}")
    print("="*80)
    print(f"Training shape: {X_tr.shape}, Test shape: {X_te.shape}")
    print()
    
    ensemble_classifiers = get_ensemble_classifiers()
    
    for model_name, model in ensemble_classifiers.items():
        print(f"  Training {model_name}...", end=' ')
        
        metrics = train_and_evaluate(
            model, X_tr, y_tr, X_te, y_te, model_name, config_name
        )
        all_results.append(metrics)
        
        print(f"✓ Acc: {metrics['Accuracy']:.4f} | F1: {metrics['F1-Score']:.4f}")
    
    print()

# ==============================================================================
# STEP 8: CREATE RESULTS DATAFRAME
# ==============================================================================

results_df = pd.DataFrame(all_results)

# ==============================================================================
# STEP 9: DISPLAY ALL RESULTS SORTED BY ACCURACY
# ==============================================================================

print("\n" + "="*80)
print("🏆 ALL RESULTS SORTED BY ACCURACY (HIGHEST TO LOWEST)")
print("="*80)
print()

results_sorted = results_df.sort_values('Accuracy', ascending=False).reset_index(drop=True)
results_sorted.index = results_sorted.index + 1

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print(results_sorted.to_string())

# ==============================================================================
# STEP 10: TOP PERFORMERS
# ==============================================================================

print("\n" + "="*80)
print("🥇 TOP 10 MODEL-CONFIGURATION COMBINATIONS")
print("="*80)
print()
print(results_sorted.head(10).to_string())

print("\n" + "="*80)
print("🥈 TOP 5 BASE MODELS")
print("="*80)
print()
base_models = results_sorted[results_sorted['Model_Type'] == 'Base'].head(5)
print(base_models.to_string())

print("\n" + "="*80)
print("🥉 TOP 3 ENSEMBLE MODELS")
print("="*80)
print()
ensemble_models = results_sorted[results_sorted['Model_Type'] == 'Ensemble'].head(3)
print(ensemble_models.to_string())

# ==============================================================================
# STEP 11: BEST MODEL PER CONFIGURATION
# ==============================================================================

print("\n" + "="*80)
print("📊 BEST MODEL PER CONFIGURATION")
print("="*80)

for config in configurations:
    config_name = config[0]
    config_results = results_df[results_df['Configuration'] == config_name]
    best_model = config_results.loc[config_results['Accuracy'].idxmax()]
    print(f"\n{config_name}:")
    print(f"  Model:      {best_model['Model']} ({best_model['Model_Type']})")
    print(f"  Accuracy:   {best_model['Accuracy']:.4f}")
    print(f"  Precision:  {best_model['Precision']:.4f}")
    print(f"  Recall:     {best_model['Recall']:.4f}")
    print(f"  F1-Score:   {best_model['F1-Score']:.4f}")
    print(f"  ROC-AUC:    {best_model['ROC-AUC']}")

# ==============================================================================
# STEP 12: BEST CONFIGURATION PER MODEL
# ==============================================================================

print("\n" + "="*80)
print("🎯 BEST CONFIGURATION PER MODEL")
print("="*80)

all_models = results_df['Model'].unique()
for model_name in sorted(all_models):
    model_results = results_df[results_df['Model'] == model_name]
    best_config = model_results.loc[model_results['Accuracy'].idxmax()]
    model_type = best_config['Model_Type']
    print(f"\n{model_name} ({model_type}):")
    print(f"  Best Config:  {best_config['Configuration']}")
    print(f"  Accuracy:     {best_config['Accuracy']:.4f}")
    print(f"  F1-Score:     {best_config['F1-Score']:.4f}")
    print(f"  ROC-AUC:      {best_config['ROC-AUC']}")

# ==============================================================================
# STEP 13: OVERALL BEST MODEL
# ==============================================================================

print("\n" + "="*80)
print("🌟 OVERALL BEST MODEL-CONFIGURATION COMBINATION")
print("="*80)
best_overall = results_df.loc[results_df['Accuracy'].idxmax()]
print(f"\nModel:         {best_overall['Model']}")
print(f"Type:          {best_overall['Model_Type']}")
print(f"Configuration: {best_overall['Configuration']}")
print(f"Accuracy:      {best_overall['Accuracy']:.4f}")
print(f"Precision:     {best_overall['Precision']:.4f}")
print(f"Recall:        {best_overall['Recall']:.4f}")
print(f"F1-Score:      {best_overall['F1-Score']:.4f}")
print(f"ROC-AUC:       {best_overall['ROC-AUC']}")

# ==============================================================================
# STEP 14: COMPARISON - BASE VS ENSEMBLE
# ==============================================================================

print("\n" + "="*80)
print("⚖️ BASE MODELS VS ENSEMBLE MODELS COMPARISON")
print("="*80)

base_avg = results_df[results_df['Model_Type'] == 'Base']['Accuracy'].mean()
ensemble_avg = results_df[results_df['Model_Type'] == 'Ensemble']['Accuracy'].mean()

print(f"\nBase Models Average Accuracy:     {base_avg:.4f}")
print(f"Ensemble Models Average Accuracy: {ensemble_avg:.4f}")
print(f"Difference:                       {ensemble_avg - base_avg:+.4f}")

if ensemble_avg > base_avg:
    print(f"\n✓ Ensemble models perform {((ensemble_avg/base_avg - 1) * 100):.2f}% better on average")
else:
    print(f"\n✗ Base models perform {((base_avg/ensemble_avg - 1) * 100):.2f}% better on average")

# ==============================================================================
# STEP 15: PIVOT TABLES
# ==============================================================================

print("\n" + "="*80)
print("📈 PIVOT TABLE: ACCURACY BY MODEL AND CONFIGURATION")
print("="*80)
pivot_accuracy = results_df.pivot_table(
    index='Model', 
    columns='Configuration', 
    values='Accuracy',
    aggfunc='first'
)
print(pivot_accuracy.to_string())

print("\n" + "="*80)
print("📈 PIVOT TABLE: F1-SCORE BY MODEL AND CONFIGURATION")
print("="*80)
pivot_f1 = results_df.pivot_table(
    index='Model', 
    columns='Configuration', 
    values='F1-Score',
    aggfunc='first'
)
print(pivot_f1.to_string())

# ==============================================================================
# STEP 16: SUMMARY STATISTICS
# ==============================================================================

print("\n" + "="*80)
print("📊 SUMMARY STATISTICS")
print("="*80)
print(f"\nTotal Experiments:        {len(results_df)}")
print(f"Base Models:              {len(results_df[results_df['Model_Type'] == 'Base'])}")
print(f"Ensemble Models:          {len(results_df[results_df['Model_Type'] == 'Ensemble'])}")
print(f"Number of Configurations: {len(configurations)}")
print(f"Number of Model Types:    {results_df['Model'].nunique()}")

print(f"\nOverall Accuracy Statistics:")
print(f"  Mean:    {results_df['Accuracy'].mean():.4f}")
print(f"  Median:  {results_df['Accuracy'].median():.4f}")
print(f"  Std Dev: {results_df['Accuracy'].std():.4f}")
print(f"  Min:     {results_df['Accuracy'].min():.4f}")
print(f"  Max:     {results_df['Accuracy'].max():.4f}")

# ==============================================================================
# STEP 17: EXPORT RESULTS
# ==============================================================================

results_df.to_csv('complete_model_comparison_results.csv', index=False)
results_sorted.to_csv('complete_model_comparison_sorted.csv', index=False)

print("\n" + "="*80)
print("💾 RESULTS EXPORTED")
print("="*80)
print("  ✓ complete_model_comparison_results.csv")
print("  ✓ complete_model_comparison_sorted.csv")
print("="*80)

PREPARING ALL CONFIGURATIONS
1. Boruta + SMOTE+Tomek...
2. PowerTransformer + SMOTE...
3. MI + SMOTE+Tomek...
4. MI only...
5. LDA...

✓ All configurations prepared!

PHASE 1: TRAINING BASE MODELS

CONFIGURATION: Boruta + SMOTE+Tomek
Training shape: (519, 10), Test shape: (75, 10)

  Training MLPClassifier... ✓ Acc: 0.9467 | F1: 0.9484
  Training SVC... ✓ Acc: 0.9333 | F1: 0.9341
  Training AdaBoostClassifier... ✓ Acc: 0.9067 | F1: 0.9062
  Training RandomForestClassifier... ✓ Acc: 0.9600 | F1: 0.9599
  Training ExtraTreesClassifier... ✓ Acc: 0.9333 | F1: 0.9333
  Training GradientBoostingClassifier... ✓ Acc: 0.9600 | F1: 0.9599
  Training XGBClassifier... ✓ Acc: 0.9333 | F1: 0.9333
  Training LGBMClassifier... ✓ Acc: 0.9333 | F1: 0.9338
  Training DecisionTreeClassifier... ✓ Acc: 0.9200 | F1: 0.9213

CONFIGURATION: PowerTransformer + SMOTE
Training shape: (525, 24), Test shape: (75, 24)

  Training MLPClassifier... ✓ Acc: 0.9200 | F1: 0.9213
  Training SVC... ✓ Acc: 0.9467 | F1: 0.947